# mediapipe를 이용하여 두 눈 인식 
#### colab 권장

In [ ]:
!pip install mediapipe

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/개인경로

In [1]:
!pip install mediapipe

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import cv2
import mediapipe as mp
import numpy as np
import matplotlib.pyplot as plt
from google.colab.patches import cv2_imshow

import time
import math
import os

mp_pose = mp.solutions.pose
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
denormalize_coordinates = mp_drawing._normalized_to_pixel_coordinates
mp_facemesh = mp.solutions.face_mesh

%matplotlib inline

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

import keras
from keras.models import Sequential
from keras.layers import Dense, Flatten
from keras.layers import Conv2D

from sklearn.model_selection import train_test_split

In [ ]:
def eyesAngle(image_path):
    mp_pose = mp.solutions.pose
    pose = mp_pose.Pose()

    image = cv2.imread(image_path)
    h, w = image.shape[:2]

    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    keypoints = pose.process(image)

    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

    lm = keypoints.pose_landmarks
    lmPose = mp_pose.PoseLandmark

    x1 = int(lm.landmark[lmPose.LEFT_EYE].x*w)  # 왼쪽 눈 x좌표
    y1 = int(lm.landmark[lmPose.LEFT_EYE].y*w)  # 왼쪽 눈 y좌표
    x2 = int(lm.landmark[lmPose.RIGHT_EYE].x*w)  # 오른쪽 눈 x좌표
    y2 = int(lm.landmark[lmPose.RIGHT_EYE].y*w)  # 오른쪽 눈 y좌표

    theta = math.acos((y2-y1)*(-y1) / (math.sqrt((x2-x1)**2+(y2-y1)**2)*y1))
    degree = int(180/math.pi) * theta

    return degree

In [ ]:
def get_eyes_ear(image_path):
    global left, right, AVG

    image = cv2.imread(image_path)[:, :, ::-1]
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)  # convert to RGB
    image = np.ascontiguousarray(image)
    imgH, imgW, _ = image.shape

    chosen_left_eye_idxs = [362, 385, 387, 263, 373, 380]
    chosen_right_eye_idxs = [33,  160, 158, 133, 153, 144]

    def distance(point_1, point_2):
        dist = sum([(i - j) ** 2 for i, j in zip(point_1, point_2)]) ** 0.5
        return dist

 
    def get_ear(landmarks, refer_idxs, frame_width, frame_height):
        try:
            coords_points = []
        for i in refer_idxs:
            lm = landmarks[i]
            coord = denormalize_coordinates(lm.x, lm.y,
                                            frame_width, frame_height)
            coords_points.append(coord)

        P2_P6 = distance(coords_points[1], coords_points[5])
        P3_P5 = distance(coords_points[2], coords_points[4])
        P1_P4 = distance(coords_points[0], coords_points[3])

        ear = (P2_P6 + P3_P5) / (2.0 * P1_P4)

    except:
        ear = 0.0
        coords_points = None

    return ear, coords_points


  def calculate_avg_ear(landmarks, left_eye_idxs, right_eye_idxs, image_w, image_h):
    left_ear, left_lm_coordinates = get_ear(
        landmarks,
        left_eye_idxs,
        image_w,
        image_h
    )
    right_ear, right_lm_coordinates = get_ear(
        landmarks,
        right_eye_idxs,
        image_w,
        image_h
    )
    Avg_EAR = (left_ear + right_ear) / 2.0

    return left_ear, right_ear, Avg_EAR, (left_lm_coordinates, right_lm_coordinates)


  custom_chosen_lmk_image = image.copy()

    with mp_facemesh.FaceMesh(refine_landmarks=True) as face_mesh:
        results = face_mesh.process(image).multi_face_landmarks

    if results:
        for face_id, face_landmarks in enumerate(results):
            landmarks = face_landmarks.landmark
            left, right, AVG, _ = calculate_avg_ear(
              landmarks,
              chosen_left_eye_idxs,
              chosen_right_eye_idxs,
              imgW,
              imgH
          )

    if max(left, right, AVG) > 0.2:
        answer = 'wake'
    else:
        answer = 'sleep'
        return answer, left, right

In [ ]:
wake_error = ['679.jpg', '88.jpg', '86.jpg', '1027.jpg', '127.jpg', '129.jpg', '168.jpg', '1147.jpg', '933.jpg', '80.jpg', '90.jpg', '16.jpg', '1105.jpg', '1097.jpg', '1100.jpg', '188.jpg', '1048.jpg', '1026.jpg', '744.jpg', '162.jpg', '889.jpg', '126.jpg', '187.jpg', '83.jpg', '124.jpg', '1198.jpg', '1025.jpg', '1068.jpg', '1183.jpg', '1175.jpg', '85.jpg', '178.jpg', '70.jpg', '1018.jpg', '1176.jpg', '89.jpg', '531.jpg', '20.jpg', '1103.jpg', '706.jpg', '985.jpg', '1099.jpg', '1102.jpg', '993.jpg', '345.jpg', '19.jpg', '1098.jpg', '87.jpg', '778.jpg', '1108.jpg', '14.jpg', '1181.jpg', '1024.jpg', '937.jpg', '1101.jpg', '134.jpg', '1017.jpg', '1022.jpg', '135.jpg', '707.jpg', '1071.jpg', '934.jpg', '170.jpg', '515.jpg', '535.jpg', '196.jpg', '1069.jpg', '1016.jpg', '1110.jpg', '82.jpg', '1002.jpg', '1095.jpg', '1094.jpg', '1021.jpg', '829.jpg', '1111.jpg', '1067.jpg', '383.jpg', '23.jpg', '858.jpg', '678.jpg', '1197.jpg', '753.jpg', '618.jpg', '191.jpg', '1072.jpg', '192.jpg', '982.jpg', '1104.jpg', '606.jpg', '1020.jpg', '732.jpg', '1145.jpg', '189.jpg', '133.jpg', '125.jpg', '99.jpg', '534.jpg', '194.jpg', '1112.jpg', '1141.jpg', '456.jpg', '43.jpg', '1127.jpg', '130.jpg', '193.jpg', '573.jpg', '65.jpg', '994.jpg', '628.jpg', '1109.jpg', '91.jpg', '69.jpg', '1140.jpg', '514.jpg', '1146.jpg', '1023.jpg', '128.jpg', '571.jpg', '743.jpg', '186.jpg', '177.jpg', '935.jpg', '116.jpg', '1070.jpg', '1001.jpg', '1003.jpg', '572.jpg', '132.jpg', '346.jpg', '337.jpg', '119.jpg', '71.jpg', '169.jpg', '115.jpg']
sleep_error = ['702.jpg', '258.jpg', '41.jpg', '700.jpg', '141.jpg', '27.jpg', '110.jpg', '312.jpg', '392.jpg', '145.jpg', '340.jpg', '45.jpg', '90.jpg', '119.jpg', '291.jpg', '51.jpg', '232.jpg', '84.jpg', '29.jpg', '334.jpg', '150.jpg', '909.jpg', '120.jpg', '352.jpg', '274.jpg', '1065.jpg', '117.jpg', '147.jpg', '96.jpg', '118.jpg', '877.jpg', '226.jpg', '148.jpg', '1068.jpg', '167.jpg', '89.jpg', '351.jpg', '857.jpg', '695.jpg', '47.jpg', '35.jpg', '323.jpg', '179.jpg', '207.jpg', '1006.jpg', '1029.jpg', '321.jpg', '115.jpg', '693.jpg', '144.jpg', '1183.jpg', '890.jpg', '1114.jpg', '197.jpg', '366.jpg', '374.jpg', '195.jpg', '318.jpg', '269.jpg', '146.jpg', '135.jpg', '922.jpg', '138.jpg', '200.jpg', '113.jpg', '52.jpg', '159.jpg', '822.jpg', '280.jpg', '37.jpg', '33.jpg', '256.jpg', '914.jpg', '210.jpg', '747.jpg', '1171.jpg', '95.jpg', '992.jpg', '628.jpg', '281.jpg', '34.jpg', '694.jpg', '102.jpg', '39.jpg', '142.jpg', '1178.jpg', '88.jpg', '698.jpg', '257.jpg', '38.jpg', '1117.jpg', '1010.jpg', '136.jpg', '751.jpg', '696.jpg', '134.jpg', '749.jpg', '101.jpg', '86.jpg', '209.jpg', '1103.jpg', '137.jpg', '818.jpg', '171.jpg', '607.jpg', '927.jpg', '220.jpg', '819.jpg', '384.jpg', '1031.jpg', '350.jpg', '91.jpg', '1165.jpg', '98.jpg', '847.jpg', '70.jpg', '46.jpg', '116.jpg', '1082.jpg', '234.jpg', '1100.jpg', '298.jpg', '177.jpg', '1107.jpg', '140.jpg', '972.jpg', '758.jpg', '1167.jpg', '166.jpg', '125.jpg', '1040.jpg', '916.jpg', '255.jpg', '348.jpg', '1009.jpg', '57.jpg', '601.jpg', '821.jpg', '48.jpg', '746.jpg', '941.jpg', '1056.jpg', '42.jpg', '381.jpg', '253.jpg', '173.jpg', '139.jpg', '254.jpg', '0.jpg', '940.jpg', '354.jpg', '993.jpg', '294.jpg', '1027.jpg', '938.jpg', '165.jpg', '1032.jpg', '30.jpg', '1033.jpg', '97.jpg', '94.jpg', '270.jpg', '156.jpg', '178.jpg', '111.jpg', '333.jpg', '915.jpg', '99.jpg', '697.jpg', '1111.jpg', '373.jpg', '1182.jpg', '353.jpg', '93.jpg', '884.jpg', '1046.jpg', '5.jpg', '944.jpg', '1044.jpg', '606.jpg', '1168.jpg', '49.jpg', '1055.jpg', '92.jpg', '180.jpg', '957.jpg', '155.jpg', '143.jpg', '114.jpg', '1118.jpg', '87.jpg', '172.jpg', '699.jpg', '85.jpg', '174.jpg', '112.jpg', '40.jpg', '997.jpg', '296.jpg', '1064.jpg', '201.jpg', '315.jpg', '371.jpg', '43.jpg', '937.jpg', '182.jpg', '920.jpg', '164.jpg', '399.jpg', '701.jpg', '265.jpg', '252.jpg', '356.jpg', '105.jpg']

In [ ]:
cnt = 0
tmp = []
for image in wake_error:
    if 80 < eyesAngle(f'wake-samples3/{image}') < 100:
        pass
    else:
        cnt += 1
        tmp.append(image)
print(cnt)
print(tmp)

In [ ]:
sorted(tmp, key=lambda x: int(x.split('.')[0]))  # 0-199 삭제..?

In [ ]:
cnt = 0
tmp = []
for image in sleep_error:
    if 80 < eyesAngle(f'sleep-samples3/{image}') < 100:
        cnt += 1
        tmp.append(image)
print(cnt)
print(tmp)

In [ ]:
sorted(tmp, key=lambda x: int(x.split('.')[0]))

### 랜드마크를 좌표로 cnn

In [ ]:
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_pose = mp.solutions.pose

IMAGE_FILES = os.listdir('wake-samples3')
BG_COLOR = (192, 192, 192)  # 회색
with mp_pose.Pose(
        static_image_mode=True,
        model_complexity=2,
        enable_segmentation=True,
        min_detection_confidence=0.5) as pose:
    wake_landmarks = []
    for idx, files in enumerate(IMAGE_FILES):
        image = cv2.imread(f'wake-samples3/{files}')
        image_height, image_width, _ = image.shape

        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        location = []
        for landmark in results.pose_world_landmarks.landmark:
            location.append([landmark.x, landmark.y, landmark.z])
        wake_landmarks.append(np.array(location))

        if idx % 100 == 0:
            print(idx)

IMAGE_FILES = os.listdir('sleep-samples3')
BG_COLOR = (192, 192, 192)  # 회색
with mp_pose.Pose(
        static_image_mode=True,
        model_complexity=2,
        enable_segmentation=True,
        min_detection_confidence=0.5) as pose:
    sleep_landmarks = []
    for idx, files in enumerate(IMAGE_FILES):
        image = cv2.imread(f'sleep-samples3/{files}')
        image_height, image_width, _ = image.shape

        results = pose.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

        location = []
        for landmark in results.pose_world_landmarks.landmark:
            location.append([landmark.x, landmark.y, landmark.z])
        sleep_landmarks.append(np.array(location))

        if idx % 100 == 0:
            print(idx)

In [ ]:
# wake: 0, sleep: 1
wake_labels = np.zeros(shape=(1202,))
sleep_labels = np.ones(shape=(1202,))

In [ ]:
print(f'wake_landmarks {np.array(wake_landmarks).shape}, wake_labels {wake_labels.shape}')
print(f'sleep_landmarks {np.array(sleep_landmarks).shape}, sleep_labels {sleep_labels.shape}')

In [ ]:
pose_landmarks = np.vstack((np.array(wake_landmarks), np.array(sleep_landmarks)))
pose_labels = np.hstack((wake_labels, sleep_labels))

In [ ]:
print(f'pose_landmarks {pose_landmarks.shape}, pose_labels {pose_labels.shape}')

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(pose_landmarks, pose_labels, test_size=0.2, random_state=156)

In [ ]:
X_train = X_train.reshape((1923, 33, 3, 1))
X_test = X_test.reshape((481, 33, 3, 1))

In [ ]:
model = Sequential()

In [ ]:
model.add(Conv2D(32, (3, 2), padding='valid', input_shape=(33, 3, 1), activation='relu'))
# (3, 2)가 가장 잘나옴

In [ ]:
model.add(layers.Flatten())
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(8, activation='relu'))
model.add(Dense(4, activation='relu'))
model.add(Dense(1, activation='sigmoid')) 

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='adam', metrics='accuracy')

In [ ]:
model.fit(X_train, y_train, epochs=10, batch_size=5)

In [ ]:
model.evaluate(X_test, y_test)  # [0.3510306179523468, 0.8274428248405457]

### 눈 추출

In [25]:
mp_drawing = mp.solutions.drawing_utils
mp_face_mesh = mp.solutions.face_mesh
drawing_spec = mp_drawing.DrawingSpec(thickness=1, circle_radius=1)
denormalize_coordinates = mp_drawing._normalized_to_pixel_coordinates

In [77]:
def getLandmarks(image_path):

    static_image_mode = True
    max_num_faces = 10
    refine_landmarks = True
    min_detection_confidence = 0.5  

    image = cv2.imread(image_path)

    with mp_face_mesh.FaceMesh(static_image_mode=static_image_mode, 
                           max_num_faces=max_num_faces, 
                           refine_landmarks=refine_landmarks, 
                           min_detection_confidence=min_detection_confidence) as face_mesh:
    results = face_mesh.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))

    if results.multi_face_landmarks == None:
        landmarks = []
    else:
        landmarks = results.multi_face_landmarks[0].landmark

    return landmarks, results

In [78]:
def distance(point_1, point_2):
    dist = sum([(i - j) ** 2 for i, j in zip(point_1, point_2)]) ** 0.5
    return dist

In [79]:
def getEyes(image, landmarks):
    r_eye_top = int(landmarks[257].y * image.shape[0])
    r_eye_left = int(landmarks[463].x * image.shape[1])
    r_eye_bottom = int(landmarks[253].y * image.shape[0])
    r_eye_right = int(landmarks[359].x * image.shape[1])
    right_eye = image[r_eye_top:r_eye_bottom, r_eye_left:r_eye_right]  

    l_eye_top = int(landmarks[27].y * image.shape[0])
    l_eye_left = int(landmarks[130].x * image.shape[1])
    l_eye_bottom = int(landmarks[23].y * image.shape[0])
    l_eye_right = int(landmarks[243].x * image.shape[1])
    left_eye = image[l_eye_top:l_eye_bottom, l_eye_left:l_eye_right]

  # open/close
  coords_points = []
    for i in [33,  160, 158, 133, 153, 144, 362, 385, 387, 263, 373, 380]:  # right: 0~5, left: 6~11
        landmark = landmarks[i]
        coord = denormalize_coordinates(landmark.x, landmark.y, image.shape[1], image.shape[0])  # x, y, w, h
        coords_points.append(coord)

    R_P2_P6 = distance(coords_points[7], coords_points[11])
    R_P3_P5 = distance(coords_points[8], coords_points[10])
    R_P1_P4 = distance(coords_points[6], coords_points[9])

    R_ear = (R_P2_P6 + R_P3_P5) / (2.0 * R_P1_P4)
    
    if R_ear >= 0.25:
        R_answer = 'open'
    else:
        R_answer = 'close'

    L_P2_P6 = distance(coords_points[1], coords_points[5])
    L_P3_P5 = distance(coords_points[2], coords_points[4])
    L_P1_P4 = distance(coords_points[0], coords_points[3])

    L_ear = (L_P2_P6 + L_P3_P5) / (2.0 * L_P1_P4)
    
    if L_ear >= 0.25:
        L_answer = 'open'
    else:
        L_answer = 'close'

    return right_eye, left_eye, R_answer, L_answer

In [43]:
image_path = 'wake-samples3/1201.jpg'
landmarks, results = getLandmarks(image_path)
image = cv2.imread(image_path)

In [44]:
rightEyeImg, leftEyeImg, rightAnswer, leftAnswer = getEyes(image, landmarks)

In [45]:
cv2_imshow(rightEyeImg)
cv2_imshow(leftEyeImg)

In [46]:
print(f'Right Eye : {rightAnswer}')
print(f'Left Eye : {leftAnswer}')

Right Eye : open
Left Eye : open


### 눈 이미지 저장

In [83]:
for image_path in os.listdir('wake-samples3'):

    landmarks, results = getLandmarks(f'wake-samples3/{image_path}')
    image = cv2.imread(f'wake-samples3/{image_path}')

    if len(landmarks) == 0:
        pass
    else:
        rightEyeImg, leftEyeImg, rightAnswer, leftAnswer = getEyes(image, landmarks)

    if rightEyeImg.shape[1] != 0:
        if rightAnswer == 'open':
            num = len(os.listdir('eyes-samples/open'))
            cv2.imwrite(f'/content/gdrive/MyDrive/개인경로/{num+1}.jpg', rightEyeImg)
        else:
            num = len(os.listdir('eyes-samples/close'))
            cv2.imwrite(f'/content/gdrive/MyDrive/개인경로/{num+1}.jpg', rightEyeImg)
    else:
        pass

    if leftEyeImg.shape[1] != 0:
        if leftAnswer == 'open':
            num = len(os.listdir('eyes-samples/open'))
            cv2.imwrite(f'/content/gdrive/MyDrive/개인경로/{num+1}.jpg', leftEyeImg)
        else:
            num = len(os.listdir('eyes-samples/close'))
            cv2.imwrite(f'/content/gdrive/MyDrive/개인경로/{num+1}.jpg', leftEyeImg)
    else:
        pass

In [84]:
for image_path in os.listdir('sleep-samples3'):

    landmarks, results = getLandmarks(f'sleep-samples3/{image_path}')
    image = cv2.imread(f'sleep-samples3/{image_path}')

    if len(landmarks) == 0:
        pass
    else:
        rightEyeImg, leftEyeImg, rightAnswer, leftAnswer = getEyes(image, landmarks)

    if rightEyeImg.shape[1] != 0:
        if rightAnswer == 'open':
            num = len(os.listdir('eyes-samples/open'))
            cv2.imwrite(f'/content/gdrive/MyDrive/개인경로/{num+1}.jpg', rightEyeImg)
        else:
            num = len(os.listdir('eyes-samples/close'))
            cv2.imwrite(f'/content/gdrive/MyDrive/개인경로/{num+1}.jpg', rightEyeImg)
    else:
        pass

    if leftEyeImg.shape[1] != 0:
        if leftAnswer == 'open':
            num = len(os.listdir('eyes-samples/open'))
            cv2.imwrite(f'/content/gdrive/MyDrive/개인경로/{num+1}.jpg', leftEyeImg)
        else:
            num = len(os.listdir('eyes-samples/close'))
            cv2.imwrite(f'/content/gdrive/MyDrive/개인경로/{num+1}.jpg', leftEyeImg)
    else:
        pass